Initialization de l'environnement pour Saturn cloud
1. **Activate `monai-dev`** ：
   ```bash
    conda env create  -f  monai-dev.yml
    conda activate monai-dev
    pip install monai timm seaborn opencv-python simpleITK
    sudo apt-get install libglib2.0-0
   ```

2. **Add environnement sur Jupyter kernel**：
   ```bash
   python -m ipykernel install --user --name=monai-dev --display-name "Python (monai-dev)"
   ```


Ce code traite des images médicales au format `.mha` et les convertit en `.npy`. Il lit les fichiers d'images et de labels, applique une compression par tranches, extrait un nombre fixe de tranches centrales, redimensionne les images (par défaut 256×256), ignore les tranches sans annotations, et enregistre les données traitées en `.npy` pour faciliter l'entraînement des modèles d'apprentissage profond.


In [21]:
# import os
# import numpy as np
# import SimpleITK as sitk
# from scipy.ndimage import zoom

# def mha_to_npy_modified(input_dir, output_dir, image_size=(256, 256), slice_axis=0, num_slices=64):
#     
#     images_path = os.path.join(input_dir, 'images')
#     labels_path = os.path.join(input_dir, 'labels')

#     if not os.path.exists(images_path) or not os.path.exists(labels_path):
#         raise FileNotFoundError("Assurez-vous que le répertoire d'entrée contient les dossiers « images » et « labels »")

#     for sub_dir in ['Dataset', 'Dataset/images', 'Dataset/labels']:
#         os.makedirs(os.path.join(output_dir, sub_dir), exist_ok=True)

#     image_files = [f for f in os.listdir(images_path) if f.endswith('.mha')]

#     for image_file in image_files:
#         img_path = os.path.join(images_path, image_file)
#         label_name = '_'.join(image_file.split('_')[:-1]) + '.mha'
#         mask_path = os.path.join(labels_path, label_name)

#         if not os.path.exists(mask_path):
#             print(f"Sauter : fichier d'étiquette non trouvé {mask_path}")
#             continue

#         img = sitk.GetArrayFromImage(sitk.ReadImage(img_path)).astype(np.float32)
#         mask = sitk.GetArrayFromImage(sitk.ReadImage(mask_path)).astype(np.int64)

#         if slice_axis >= len(img.shape):
#             raise ValueError(f"Axe de la tranche {slice_axis} au-delà de la dimension de l'image {img.shape}")

#         # Profondeur de la compression : les tranches paires indexées sont supprimées
#         compressed_img = np.take(img, indices=range(0, img.shape[slice_axis], 2), axis=slice_axis)
#         compressed_mask = np.take(mask, indices=range(0, mask.shape[slice_axis], 2), axis=slice_axis)

#         total_slices = compressed_img.shape[slice_axis]
#         if total_slices < num_slices:
#             raise ValueError(f"La profondeur de l'image compressée {total_slices} est inférieure au nombre de tranches requises {num_slices}")

#         start_idx = (total_slices - num_slices) // 2
#         end_idx = start_idx + num_slices

#         selected_img_slices = np.take(compressed_img, indices=range(start_idx, end_idx), axis=slice_axis)
#         selected_mask_slices = np.take(compressed_mask, indices=range(start_idx, end_idx), axis=slice_axis)

#         saved_slices = 0
#         for i in range(num_slices):
#             img_slice = selected_img_slices[i]
#             mask_slice = selected_mask_slices[i]

#             if np.sum(mask_slice) == 0:
#                 continue  # Ignorer les tranches sans annotation

#             zoom_factors = [image_size[0] / img_slice.shape[0], image_size[1] / img_slice.shape[1]]
#             img_slice_resized = zoom(img_slice, zoom_factors, order=1)  
#             mask_slice_resized = zoom(mask_slice, zoom_factors, order=0)  

#             output_name = f"{os.path.splitext(image_file)[0]}_slice_{start_idx + i}"
#             np.save(os.path.join(output_dir, 'Dataset/images', f"{output_name}_data.npy"), img_slice_resized)
#             np.save(os.path.join(output_dir, 'Dataset/labels', f"{output_name}_gt_sparse.npy"), mask_slice_resized)

#             saved_slices += 1 
#         print(f"Traitée : {fichier_image}, total { tranches_sauvées} tranches sauvées.")
#     print("Traitement terminé.")
# input_dir = './data/Dataset'
# output_dir = './processed_data'

# mha_to_npy_modified(input_dir, output_dir, image_size=(256, 256), slice_axis=0, num_slices=64)


Le but de ce code est de nous aider à vérifier si les tranches d'images médicales au format `.npy` sont correctes. Il choisira aléatoirement une tranche et son étiquette correspondante dans le jeu de données et l'affichera en utilisant `matplotlib`.

In [22]:
# import os
# import numpy as np
# import matplotlib.pyplot as plt
# import random

# def visualize_random_slice(data_dir):

#     image_dir = os.path.join(data_dir, 'Dataset/images')
#     label_dir = os.path.join(data_dir, 'Dataset/labels')

#     if not os.path.exists(image_dir) or not os.path.exists(label_dir):
#         raise FileNotFoundError("Assurez-vous que le répertoire de données contient les dossiers « Dataset/images » et « Dataset/labels ».")

#     image_files = [f for f in os.listdir(image_dir) if f.endswith('_data.npy')]
#     label_files = [f for f in os.listdir(label_dir) if f.endswith('_gt_sparse.npy')]

#     if not image_files or not label_files:
#         raise FileNotFoundError("Impossible de trouver des fichiers .npy, vérifiez le répertoire d'entrée.")

#     random_file = random.choice(image_files)
#     corresponding_label_file = random_file.replace('_data.npy', '_gt_sparse.npy')

#     if corresponding_label_file not in label_files:
#         raise FileNotFoundError(f"Fichier d'étiquettes correspondant non trouvé :  {corresponding_label_file}")

#     image_path = os.path.join(image_dir, random_file)
#     label_path = os.path.join(label_dir, corresponding_label_file)

#     image = np.load(image_path)
#     label = np.load(label_path)

#     plt.figure(figsize=(10, 5))

#     plt.subplot(1, 2, 1)
#     plt.imshow(image, cmap='gray')
#     plt.title('Image')
#     plt.axis('off')

#     plt.subplot(1, 2, 2)
#     plt.imshow(label, cmap='viridis')
#     plt.title('Label')
#     plt.axis('off')

#     plt.tight_layout()
#     plt.show()

# processed_data_dir = './processed_data'

# visualize_random_slice(processed_data_dir)


Ce code est utilisé pour vérifier si le GPU (CUDA) est disponible pour PyTorch et fournit des informations sur le GPU ou, s'il n'est pas disponible, indique qu'il tourne actuellement sur un CPU.

In [23]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

# If available, display GPU details
if cuda_available:
    print("Number of GPU devices:", torch.cuda.device_count())
    print("Current GPU device index:", torch.cuda.current_device())
    print("Current GPU device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. PyTorch is running on CPU.")

CUDA Available: False
CUDA is not available. PyTorch is running on CPU.


Ce code est utilisé pour importer les bibliothèques et les modules nécessaires à l'entraînement des modèles d'apprentissage profond, y compris des fonctions telles que le chargement de données, la construction de modèles, l'optimisation, l'évaluation, etc. en préparation du processus d'entraînement ultérieur.

In [24]:

import argparse
import os
import sys
import time
from collections import OrderedDict
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from skimage import io
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from tensorboardX import SummaryWriter
#from dataset import *
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm

import cfg
import function
from conf import settings
#from models.discriminatorlayer import discriminator
from dataset import *
from utils import *

ModuleNotFoundError: No module named 'tensorboardX'

Ce code parcourt le chargeur de données `nice_train_loader`, affiche les informations sur les lots d'images et d'étiquettes, et visualise certaines d'entre elles avec `matplotlib`. Cela permet de vérifier que les données sont correctement chargées et au bon format pour l'entraînement du modèle.

In [5]:
nice_train_loader, nice_test_loader = get_dataloader(args)

# # Itération sur plusieurs lots, impression et visualisation
# for i, batch in enumerate(nice_train_loader):
#     print(f"Batch {i}:")
#     print(f"Image Batch Shape: {batch['image'].shape}, Range: [{batch['image'].min()}, {batch['image'].max()}]")
#     print(f"Label Batch Shape: {batch['label'].shape}, Unique Values: {torch.unique(batch['label'])}")

#     print(f"Files or Patient IDs in Batch {i}:")
#     for filename_or_id in batch['image_meta_dict']['filename_or_obj']:
#         print(filename_or_id)

#     plt.figure(figsize=(10, 5))
#     plt.subplot(1, 2, 1)
#     plt.title(f"Batch {i} - Image")
#     plt.imshow(batch['image'][0, 0].cpu().numpy(), cmap='gray')  # 假设第0张图像是单通道
#     plt.subplot(1, 2, 2)
#     plt.title(f"Batch {i} - Label")
#     plt.imshow(batch['label'][0].argmax(dim=0).cpu().numpy(), cmap='nipy_spectral')  # 假设是 one-hot 编码
#     plt.show()

#     if i == 2:  
#         break


Training Patients: ['ToothFairy2F_007_0000', 'ToothFairy2F_054_0000', 'ToothFairy2F_046_0000', 'ToothFairy2F_024_0000', 'ToothFairy2F_010_0000', 'ToothFairy2F_011_0000', 'ToothFairy2F_039_0000', 'ToothFairy2F_001_0000', 'ToothFairy2F_013_0000', 'ToothFairy2F_009_0000', 'ToothFairy2F_032_0000', 'ToothFairy2F_018_0000', 'ToothFairy2F_006_0000', 'ToothFairy2F_030_0000', 'ToothFairy2F_051_0000', 'ToothFairy2F_031_0000', 'ToothFairy2F_034_0000', 'ToothFairy2F_003_0000', 'ToothFairy2F_027_0000', 'ToothFairy2F_029_0000', 'ToothFairy2F_028_0000', 'ToothFairy2F_022_0000', 'ToothFairy2F_040_0000', 'ToothFairy2F_044_0000', 'ToothFairy2F_059_0000', 'ToothFairy2F_008_0000', 'ToothFairy2F_021_0000', 'ToothFairy2F_058_0000', 'ToothFairy2F_041_0000', 'ToothFairy2F_050_0000', 'ToothFairy2F_014_0000', 'ToothFairy2F_066_0000', 'ToothFairy2F_067_0000', 'ToothFairy2F_065_0000', 'ToothFairy2F_033_0000', 'ToothFairy2F_036_0000', 'ToothFairy2F_037_0000', 'ToothFairy2F_062_0000', 'ToothFairy2F_026_0000', 'Toot

Ce code initialise et charge un modèle de réseau neuronal avec des poids pré-entraînés. Il configure le GPU, charge les poids du modèle SAM et d'un adaptateur spécifique à la tâche, et ajuste la gestion des poids pour l'entraînement distribué si nécessaire. Ensuite, il initialise un optimiseur Adam, un programme de diminution du taux d'apprentissage et, si disponible, recharge un modèle pré-entraîné pour reprendre l'entraînement à partir d'un point de contrôle sauvegardé. Enfin, il configure le journal des logs pour suivre l'entraînement.


In [6]:
args = cfg.parse_args()
args.sam_ckpt = './sam_vit_b_01ec64.pth'
GPUdevice = torch.device('cuda', args.gpu_device)

net = get_network(args, args.net, use_gpu=args.gpu, gpu_device=GPUdevice, distribution=args.distributed)
# print(f"image encoder= {net.image_encoder}")
# print(f"mask decoder={net.mask_decoder}")
# print(f"prompt encoder= {net.prompt_encoder}")


assert os.path.exists(args.sam_ckpt), f"File not found: {args.sam_ckpt}"
print(f"Checkpoint path: {args.sam_ckpt} exists, loading...")

sam_weights = args.sam_ckpt
with open(sam_weights, "rb") as f:
    state_dict = torch.load(f)
    print(f"Loaded SAM weights with {len(state_dict)} keys from: {sam_weights}")

new_state_dict = {k: v for k, v in state_dict.items() if k in net.state_dict() and net.state_dict()[k].shape == v.shape}
print(f"Number of matched keys loaded into the network: {len(new_state_dict)}")
net.load_state_dict(new_state_dict, strict=False)

adapter_path = 'CBCT.pth'
checkpoint_file = os.path.join(adapter_path)

assert os.path.exists(checkpoint_file), f"File not found: {checkpoint_file}"
print(f"Adapter checkpoint path: {checkpoint_file} exists, loading...")

loc = 'cuda:{}'.format(args.gpu_device)
checkpoint = torch.load(checkpoint_file, map_location=loc)
print(f"Loaded Adapter weights with {len(checkpoint['state_dict'])} keys from: {checkpoint_file}")

if args.distributed != 'none':
    from collections import OrderedDict
    new_adapter_state_dict = OrderedDict()
    for k, v in checkpoint['state_dict'].items():
        name = 'module.' + k
        new_adapter_state_dict[name] = v
    print("Loaded adapter weights for distributed training.")
else:
    new_adapter_state_dict = checkpoint['state_dict']

net.load_state_dict(new_adapter_state_dict, strict=False)
print("Task-specific adapter weights successfully loaded.")

if args.pretrain:
    weights = torch.load(args.pretrain)
    net.load_state_dict(weights,strict=False)

optimizer = optim.Adam(net.parameters(), lr=args.lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay

'''load pretrained model'''
if args.weights != 0:
    print(f'=> resuming from {args.weights}')
    assert os.path.exists(args.weights)
    checkpoint_file = os.path.join(args.weights)
    assert os.path.exists(checkpoint_file)
    loc = 'cuda:{}'.format(args.gpu_device)
    checkpoint = torch.load(checkpoint_file, map_location=loc)
    start_epoch = checkpoint['epoch']
    best_tol = checkpoint['best_tol']

    net.load_state_dict(checkpoint['state_dict'],strict=False)
    # optimizer.load_state_dict(checkpoint['optimizer'], strict=False)

    args.path_helper = checkpoint['path_helper']
    logger = create_logger(args.path_helper['log_path'])
    print(f'=> loaded checkpoint {checkpoint_file} (epoch {start_epoch})')

args.path_helper = set_log_dir('logs', args.exp_name)
logger = create_logger(args.path_helper['log_path'])
logger.info(args)


Unknown arguments: ['-f', '/home/jovyan/.local/share/jupyter/runtime/kernel-7e488bbf-ed8a-4b91-801d-42b3628a77f7.json']


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Checkpoint path: ./sam_vit_b_01ec64.pth exists, loading...


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Loaded SAM weights with 314 keys from: ./sam_vit_b_01ec64.pth
Number of matched keys loaded into the network: 279
Adapter checkpoint path: CBCT.pth exists, loading...


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
Namespace(net='sam', baseline='unet', encoder='default', seg_net='transunet', mod='sam_lora', exp_name='msa_test_isic', type='map', vis=10, reve

Loaded Adapter weights with 144 keys from: CBCT.pth
Task-specific adapter weights successfully loaded.


In [7]:
'''checkpoint path and tensorboard'''
# iter_per_epoch = len(Glaucoma_training_loader)
checkpoint_path = os.path.join(settings.CHECKPOINT_PATH, args.net, settings.TIME_NOW)
#use tensorboard
if not os.path.exists(settings.LOG_DIR):
    os.mkdir(settings.LOG_DIR)
writer = SummaryWriter(log_dir=os.path.join(
        settings.LOG_DIR, args.net, settings.TIME_NOW))
# input_tensor = torch.Tensor(args.b, 3, 256, 256).cuda(device = GPUdevice)
# writer.add_graph(net, Variable(input_tensor, requires_grad=True))

#create checkpoint folder to save model
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{net}-{epoch}-{type}.pth')

Ce code met en œuvre la boucle d'entraînement d'un modèle de segmentation basé sur SAM. Il effectue les étapes suivantes :
- Entraîne le modèle pendant un nombre défini d'époques.
- Évalue périodiquement les performances sur un ensemble de test en calculant les métriques IOU et Dice pour chaque classe.
- Enregistre les pertes d'entraînement et de validation dans les logs.
- Sauvegarde le modèle si la meilleure valeur de Dice est atteinte.
- Utilise `tensorboardX` pour suivre les performances et `logger` pour journaliser les résultats.

Cela permet d'entraîner et d'ajuster le modèle tout en conservant les meilleurs poids obtenus.


In [ ]:
'''begain training'''
best_acc = 0.0
best_tol = 1e4
best_dice = 0.0
for epoch in range(settings.EPOCH):
    # if epoch and epoch < 5:
    #     if args.dataset != 'REFUGE':
    #         tol, (eiou, edice) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
    #         logger.info(f'Total score: {tol}, IOU: {eiou}, DICE: {edice} || @ epoch {epoch}.')
    #     else:
    #         tol, (eiou_cup, eiou_disc, edice_cup, edice_disc) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
    #         logger.info(f'Total score: {tol}, IOU_CUP: {eiou_cup}, IOU_DISC: {eiou_disc}, DICE_CUP: {edice_cup}, DICE_DISC: {edice_disc} || @ epoch {epoch}.')
    if epoch and epoch < 30:
        if args.dataset != 'REFUGE':
        
            tol, avg_iou, avg_dice, iou_per_class, dice_per_class = function.validation_sam(args, nice_test_loader, epoch, net, writer)
            logger.info(f'Total loss: {tol}, Average IOU: {avg_iou}, Average Dice: {avg_dice}')
            
            for i, (iou, dice) in enumerate(zip(iou_per_class, dice_per_class)):
                logger.info(f'Class {i}: IOU={iou:.4f}, Dice={dice:.4f}')

    net.train()
    time_start = time.time()
    loss = function.train_sam(args, net, optimizer, nice_train_loader, epoch, writer, vis = args.vis)
    logger.info(f'Train loss: {loss} || @ epoch {epoch}.')
    time_end = time.time()
    print('time_for_training ', time_end - time_start)

    net.eval()
    if epoch and epoch % args.val_freq == 0 or epoch == settings.EPOCH-1:
        # if args.dataset != 'REFUGE':
        #     tol, (eiou, edice) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
        #     logger.info(f'Total score: {tol}, IOU: {eiou}, DICE: {edice} || @ epoch {epoch}.')
        # else:
        #     tol, (eiou_cup, eiou_disc, edice_cup, edice_disc) = function.validation_sam(args, nice_test_loader, epoch, net, writer)
        #     logger.info(f'Total score: {tol}, IOU_CUP: {eiou_cup}, IOU_DISC: {eiou_disc}, DICE_CUP: {edice_cup}, DICE_DISC: {edice_disc} || @ epoch {epoch}.')
        if args.dataset != 'REFUGE':
           
            tol, avg_iou, avg_dice, iou_per_class, dice_per_class = function.validation_sam(args, nice_test_loader, epoch, net, writer)
            logger.info(f'Total loss: {tol}, Average IOU: {avg_iou}, Average Dice: {avg_dice}')
          
            for i, (iou, dice) in enumerate(zip(iou_per_class, dice_per_class)):
                logger.info(f'Class {i}: IOU={iou:.4f}, Dice={dice:.4f}')
        if args.distributed != 'none':
            sd = net.module.state_dict()
        else:
            sd = net.state_dict()

        if avg_dice > best_dice:
            best_dice = avg_dice  
            best_tol = tol
            is_best = True

            save_checkpoint({
            'epoch': epoch + 1,
            'model': args.net,
            'state_dict': sd,
            'optimizer': optimizer.state_dict(),
            'best_tol': best_dice,
            'path_helper': args.path_helper,
        }, is_best, args.path_helper['ckpt_path'], filename="best_dice_checkpoint.pth")
        else:
            is_best = False

writer.close()

Epoch 0: 100%|██████████| 50/50 [02:31<00:00,  3.04s/img, loss (batch)=1.3] 
Train loss: 1.301276683807373 || @ epoch 0.


time_for_training  151.83425951004028


Total loss: 1.3302979469299316, Average IOU: 0.728339259093796, Average Dice: 0.47537517151745456
Class 0: IOU=0.9179, Dice=0.9570
Class 1: IOU=0.0585, Dice=0.1095
Class 2: IOU=0.0000, Dice=0.0000
Class 3: IOU=0.0002, Dice=0.0005
Class 4: IOU=0.1202, Dice=0.0000
Class 5: IOU=0.9971, Dice=0.9538
Class 6: IOU=0.9933, Dice=0.8923
Class 7: IOU=0.0131, Dice=0.0257
Class 8: IOU=0.9615, Dice=0.8462
Class 9: IOU=0.6558, Dice=0.2615
Class 10: IOU=0.9346, Dice=0.6462
Class 11: IOU=0.8788, Dice=0.2308
Class 12: IOU=0.9750, Dice=0.8308
Class 13: IOU=0.9375, Dice=0.6154
Class 14: IOU=0.9625, Dice=0.7846
Class 15: IOU=0.9644, Dice=0.8154
Class 16: IOU=0.9587, Dice=0.6000
Class 17: IOU=0.9856, Dice=0.9231
Class 18: IOU=0.9990, Dice=0.9846
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=0.9962, Dice=0.9385
Class 21: IOU=0.9231, Dice=0.3846
Class 22: IOU=0.9490, Dice=0.6462
Class 23: IOU=0.9375, Dice=0.5077
Class 24: IOU=0.9394, Dice=0.5231
Class 25: IOU=0.9317, Dice=0.4154
Class 26: IOU=0.9471, Dice=0

time_for_training  152.65892839431763


Total loss: 1.2837419509887695, Average IOU: 0.5737960105833175, Average Dice: 0.21181574004099066
Class 0: IOU=0.9179, Dice=0.9570
Class 1: IOU=0.0604, Dice=0.1128
Class 2: IOU=0.0000, Dice=0.0000
Class 3: IOU=0.0002, Dice=0.0005
Class 4: IOU=0.1115, Dice=0.0000
Class 5: IOU=0.8952, Dice=0.2000
Class 6: IOU=0.9481, Dice=0.3538
Class 7: IOU=0.0133, Dice=0.0262
Class 8: IOU=0.9212, Dice=0.3692
Class 9: IOU=0.3342, Dice=0.0174
Class 10: IOU=0.8327, Dice=0.1846
Class 11: IOU=0.6577, Dice=0.0000
Class 12: IOU=0.9163, Dice=0.2154
Class 13: IOU=0.9212, Dice=0.4308
Class 14: IOU=0.9106, Dice=0.2769
Class 15: IOU=0.9010, Dice=0.2769
Class 16: IOU=0.8885, Dice=0.2000
Class 17: IOU=0.9394, Dice=0.4000
Class 18: IOU=0.9712, Dice=0.5538
Class 19: IOU=0.9510, Dice=0.3846
Class 20: IOU=0.9644, Dice=0.4615
Class 21: IOU=0.8308, Dice=0.0923
Class 22: IOU=0.8644, Dice=0.1385
Class 23: IOU=0.8510, Dice=0.1231
Class 24: IOU=0.8798, Dice=0.1538
Class 25: IOU=0.8846, Dice=0.1538
Class 26: IOU=0.9317, Dice=

...

Total loss: 0.7834171652793884, Average IOU: 0.6523402061914546, Average Dice: 0.3996662804021383
Class 0: IOU=0.9474, Dice=0.9730
Class 1: IOU=0.5622, Dice=0.7279
Class 2: IOU=1.0000, Dice=1.0000
Class 3: IOU=0.1088, Dice=0.0062
Class 4: IOU=0.1087, Dice=0.0099
Class 5: IOU=1.0000, Dice=1.0000
Class 6: IOU=1.0000, Dice=1.0000
Class 7: IOU=0.7489, Dice=0.8579
Class 8: IOU=0.5511, Dice=0.0539
Class 9: IOU=0.1617, Dice=0.0794
Class 10: IOU=0.7900, Dice=0.1562
Class 11: IOU=0.7146, Dice=0.1369
Class 12: IOU=0.8289, Dice=0.2200
Class 13: IOU=0.7878, Dice=0.1761
Class 14: IOU=0.8837, Dice=0.3077
Class 15: IOU=0.8981, Dice=0.4462
Class 16: IOU=0.6317, Dice=0.0779
Class 17: IOU=0.9856, Dice=0.9231
Class 18: IOU=1.0000, Dice=1.0000
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=1.0000, Dice=1.0000
Class 21: IOU=0.7325, Dice=0.1287
Class 22: IOU=0.7592, Dice=0.2052
Class 23: IOU=0.6650, Dice=0.1503
Class 24: IOU=0.6120, Dice=0.0336
Class 25: IOU=0.7237, Dice=0.0201
Class 26: IOU=0.4397, Dice=0

time_for_training  154.46863865852356


Total loss: 0.7767809629440308, Average IOU: 0.6731684219554951, Average Dice: 0.42566391241627055
Class 0: IOU=0.9495, Dice=0.9742
Class 1: IOU=0.5734, Dice=0.7381
Class 2: IOU=1.0000, Dice=1.0000
Class 3: IOU=0.1061, Dice=0.0200
Class 4: IOU=0.1096, Dice=0.0148
Class 5: IOU=1.0000, Dice=1.0000
Class 6: IOU=1.0000, Dice=1.0000
Class 7: IOU=0.7612, Dice=0.8662
Class 8: IOU=0.5280, Dice=0.0077
Class 9: IOU=0.2139, Dice=0.0880
Class 10: IOU=0.4258, Dice=0.0135
Class 11: IOU=0.7786, Dice=0.2514
Class 12: IOU=0.9174, Dice=0.6165
Class 13: IOU=0.8760, Dice=0.3611
Class 14: IOU=0.8991, Dice=0.3082
Class 15: IOU=0.9010, Dice=0.4154
Class 16: IOU=0.7801, Dice=0.1773
Class 17: IOU=0.9856, Dice=0.9231
Class 18: IOU=1.0000, Dice=1.0000
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=1.0000, Dice=1.0000
Class 21: IOU=0.7864, Dice=0.1721
Class 22: IOU=0.7810, Dice=0.1945
Class 23: IOU=0.7454, Dice=0.2468
Class 24: IOU=0.6778, Dice=0.0756
Class 25: IOU=0.8601, Dice=0.2213
Class 26: IOU=0.6358, Dice=

time_for_training  153.99381351470947


Total loss: 0.772485613822937, Average IOU: 0.6560378526298601, Average Dice: 0.4067810293276476
Class 0: IOU=0.9499, Dice=0.9744
Class 1: IOU=0.5773, Dice=0.7402
Class 2: IOU=1.0000, Dice=1.0000
Class 3: IOU=0.1056, Dice=0.0200
Class 4: IOU=0.1128, Dice=0.0054
Class 5: IOU=1.0000, Dice=1.0000
Class 6: IOU=1.0000, Dice=1.0000
Class 7: IOU=0.7591, Dice=0.8648
Class 8: IOU=0.4924, Dice=0.0076
Class 9: IOU=0.1998, Dice=0.0883
Class 10: IOU=0.5039, Dice=0.0265
Class 11: IOU=0.7471, Dice=0.1175
Class 12: IOU=0.8682, Dice=0.3607
Class 13: IOU=0.8237, Dice=0.2890
Class 14: IOU=0.8858, Dice=0.2639
Class 15: IOU=0.8817, Dice=0.4000
Class 16: IOU=0.5903, Dice=0.0631
Class 17: IOU=0.9837, Dice=0.9231
Class 18: IOU=1.0000, Dice=1.0000
Class 19: IOU=1.0000, Dice=1.0000
Class 20: IOU=1.0000, Dice=1.0000
Class 21: IOU=0.7528, Dice=0.1442
Class 22: IOU=0.7871, Dice=0.3003
Class 23: IOU=0.7100, Dice=0.2051
Class 24: IOU=0.6398, Dice=0.0680
Class 25: IOU=0.6822, Dice=0.0152
Class 26: IOU=0.3786, Dice=0.